In [1]:
import numpy as np
from time import time
from matplotlib import pyplot as plt
import pandas as pd

import scipy.sparse as scs
from sklearn.utils.extmath import randomized_svd

Rekurencyjna kompresja

In [72]:
class Node:
    def __init__(self, t_min, t_max, s_min, s_max):
        self.t_min = t_min
        self.t_max = t_max
        
        self.s_min = s_min
        self.s_max = s_max
        
        self.rank = None
        
        #[A11, A12]
        #[A21, A22]
        
        self.U = None
        self.S = None
        self.VT = None
        
        self.zeros = False
        
        
        self.children = []
        
        


In [27]:
def is_admissible(matrix, S, r, eps):
    if min(matrix.shape) <= r+1:
        return True
    
    if S[-1] < eps:
        return True
    
    return False     

In [73]:
def create_tree(matrix, r, eps):
    n, m = matrix.shape

    
    def create_tree_r(t_min, t_max, s_min, s_max):
        nonlocal matrix, r, eps
        
        if not np.any(matrix[t_min:t_max, s_min:s_max]):
            v = Node(t_min, t_max, s_min, s_max)
            v.rank = 0
            v.zeros = True
            return v
    
        U,S,V = randomized_svd(matrix[t_min:t_max, s_min:s_max], r+1)
#dodać odcinanie wa
#         S = S[S >= eps]  
#         rank = len(S)
        
#         U = U[:rank, :]
#         V = V[:, :rank]
        rank = r
        if is_admissible(matrix[t_min:t_max, s_min:s_max], S, r, eps):
            v = Node(t_min, t_max, s_min, s_max) #compress matrix
            v.rank = rank
            v.U = U
            v.S = S
            v.V = V
            
        else:
            v = Node(t_min, t_max, s_min, s_max)
            
            t_newmax = (t_min + t_max)//2
            s_newmax = (s_min + s_max)//2
            
            v1 = create_tree_r(t_min, t_newmax, s_min, s_newmax)
            v2 = create_tree_r(t_min, t_newmax, s_newmax, s_max)
            v3 = create_tree_r(t_newmax, t_max, s_min, s_newmax)
            v4 = create_tree_r(t_newmax, t_max, s_newmax, s_max)
            
            v.children = [v1, v2, v3, v4]
        return v
    return create_tree_r(0, n, 0, m)

In [74]:
def decompress(node):
    if node.rank is not None:
        if node.rank > 0:
            return node.U@np.diag(node.S)@node.V
        else:
            return np.zeros((node.t_max - node.t_min, node.s_max - node.s_min))
    else:
        return np.vstack(
            (
                np.hstack((decompress(node.children[0]), decompress(node.children[1]))),
                np.hstack((decompress(node.children[2]), decompress(node.children[3]))),
            )
        )
    
    
        

In [19]:
a = scs.random(4,4,density = 0.25)
a = a.toarray()
print(a)

array([[0.        , 0.        , 0.53407934, 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.30765608, 0.        , 0.        , 0.79540715],
       [0.        , 0.        , 0.        , 0.87281031]])

In [75]:
tree = create_tree(a, 1, 10**(-8))
print(decompress(tree))

[[0.         0.         0.53407934 0.        ]
 [0.         0.         0.         0.        ]
 [0.30765608 0.         0.         0.79540715]
 [0.         0.         0.         0.87281031]]


In [77]:
print(np.allclose(a, decompress(tree)))

True
